# Description
This example is a simple demonstration of how to run a Sesam Core fatigue hotspot example using OneWorkflow locally and in the cloud.

In [2]:
from oneWorkflowToolBox import *
from pathlib import Path
# local workspace, all results will be put here after local or cloud runs
# location of common files for all analysis, has to be below workspacePath and in the folder names CommonFiles
root_folder = os.getcwd()
workspacePath = Path(root_folder, 'Workspace')
workspaceId = "SesamCoreExample"
cloud_run = False
#If running locally the code below will also start the local workflow host.
workflow_client = one_workflow_client(workspaceId, workspacePath, cloud_run,
                                      tmp=r"c:\OneWorkflowTemp", platform=Platform.Windows, max_cores=5)

if (cloud_run):
    workflow_client.login()
workflow_client.upload_common_files(FileOptions(max_size_bytes=524_000,patterns=["**/*.*"], overwrite=True))
# max number of threads used when running locally


NameError: name 'testOptional' is not defined

In [ ]:
import pandas as pd
import os
from dnv.oneworkflow import  ParallelWork
from oneWorkflowToolBox import run_workflow_async
from dnv.onecompute.flowmodel import WorkUnit
from dnv.sesam.sesam_core_command import *
from dnv.oneworkflow import PythonCommand, CompositeExecutableCommand
import json
import shutil

# we must delete existing results locally before generating new results
local__result_path = Path(workspacePath, workflow_client.results_directory)
if os.path.isdir(local__result_path):
    shutil.rmtree(local__result_path) 

parallel_work = ParallelWork()
for index in range(1,14): # iterating over two simple cases, they now will do the same analysis
    loadcase_folder_name = f"LoadCase{index}"
    result_folder_lc = os.path.join(workflow_client.results_directory, loadcase_folder_name)
    python_copy_command = PythonCommand(
        directory=workflow_client.common_directory,
        filename="copyfiles.py",
        working_dir=result_folder_lc)
    core_command = SesamCoreCommand(command = "fatigue",input_file_name= "Specimen1_input.json", options = "-hs")
    cmd = CompositeExecutableCommand([python_copy_command, core_command], result_folder_lc)
    work_unit = (WorkUnit(cmd, loadcase_folder_name)
                 .input_directory(result_folder_lc)
                 .output_directory(result_folder_lc, include_files=["**/*.txt",  "**/*.MLG", "**/*.lis", "**/*.sin"]))
    parallel_work.WorkItems.append(work_unit)
downloadOptions = FileOptions(max_size_bytes=11124_000,patterns=["**/*.txt", "**/*.lis", "**/*.MLG"])
job = workflow_client.create_job(parallel_work)
#for debugging
#job_json = json.dumps(job, default=lambda o: o.encode(), indent=4)
#print(job_json)
await run_workflow_async(job, workflow_client, downloadOptions)
